In [132]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score,KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [ ]:
# Adatok beolvasása az állományból
data = pd.read_csv('C:/Users/Tamás/Desktop/Gazdinfó/Allamvizsga_masodikfelvonas/AllStatistics1_validation.csv')

# A "Minute Played" oszlopból eltávolítani a "'" karaktert, és átkonvertálni "int"-be 
data['Minutes Played'] = data['Minutes Played'].str.replace("'", "").astype('int64')

# A kategorikus változók numerikussá alakítása One-Hot Encoding-el
data = pd.get_dummies(data, columns=["Position"], dtype=int)

# Új statisztikai adatok behozatala
data["Duel Win Rate"] = (data["Duels won"] / data["Duels"]).round(2)
data["Ground Duel Win Rate"] = (data["Ground Duels won"] / data["Ground Duels"]).round(2)
data["Aerial Duel Win Rate"] = (data["Aerial Duels won"] / data["Aerial Duels"]).round(2)
data["Tackel Success Rate"] = (data["Total tackels"] / (data["Total tackels"] + data["Dribbled past"])).round(2)
data["Possession Loss Rate"] = (data["Possession lost"] / data["Touches"]).round(2)
data["Successful Long Ball Rate"] = (data["Succesfull Long balls"] / data["Long balls attempts"]).round(2)
data["Successful Cross Rate"] = (data["Succesfull Crosses"] / data["Crosses attempts"]).round(2)
data["Interception Efficiency"] = (data["Interceptions"] / data["Defensive actions"]).round(2)
data["Clearance Efficiency"] = (data["Clearances"] / data["Defensive actions"]).round(2)
data["Foul Rate"] = (data["Fouls"] / data["Duels"]).round(2)

# A módosított adatok mentése vissza CSV-be (UTF-8-BOM-mal, hogy az ékezetek is megmaradjanak)
data.to_csv('C:/Users/Tamás/Desktop/Gazdinfó/Allamvizsga_masodikfelvonas/AllStatistics1_validation.csv', index=False, encoding='utf-8-sig')

In [163]:
# Az összes NaN érték lecserélése 0-ra
data.fillna(0, inplace=True)

# A módosított adatok mentése vissza CSV-be (UTF-8-BOM-mal, hogy az ékezetek is megmaradjanak)
data.to_csv('C:/Users/Tamás/Desktop/Gazdinfó/Allamvizsga_masodikfelvonas/AllStatistics1_validation.csv', index=False, encoding='utf-8-sig')

In [164]:
# Játékos oszlopok azonosítása
allPlayers = set(data["Player"].unique())
playerColumns = [col for col in data.columns if col in allPlayers]

statisticsColumns = data.columns.difference(playerColumns + ["MatchID", "Team", "Player", "Notes", "Defence notes", "Pass notes"])
statisticsColumns = [col for col in statisticsColumns if data[col].dtype in [int, float]]

# Meccsenként a játékosok szétválogatása csapatokra minden MatchID-re
teamInfoPerMatch = {}
allMatchIDs = data["MatchID"].unique()

for matchID in allMatchIDs:
    matchData = data[data["MatchID"] == matchID]
    team1 = set()
    team2 = set()
    playersInMatch = matchData["Player"].tolist()
    matchPlayerColumns = [col for col in data.columns if col in playersInMatch]

    for _, row in matchData.iterrows():
        playerName = row["Player"]
        for teamMate in matchPlayerColumns:
            value = row[teamMate]
            if value == matchID:
                team1.add(playerName)
            elif value == -matchID:
                team2.add(playerName)

    teamInfoPerMatch[matchID] = {
        "team1": sorted(team1),
        "team2": sorted(team2)
    }

In [165]:
# Aggregált csapatstatisztikák hozzárendelése minden játékoshoz
agregatedRows = []

for matchID in allMatchIDs:
    matchData = data[data["MatchID" ] == matchID]
    teams = teamInfoPerMatch[matchID]
    team1 = teams["team1"]
    team2 = teams["team2"]

    team1Statistics = matchData[matchData["Player"].isin(team1)][statisticsColumns]
    team2Statistics = matchData[matchData["Player"].isin(team2)][statisticsColumns]

    team1Summary = {
        "team_total_goals": team1Statistics["Goals"].sum(),
        "team_total_assists": team1Statistics["Assists"].sum(),
        "team_total_shots": team1Statistics["Shots on target"].sum(),
        "team_avg_xG": team1Statistics["Expected goals (xG)"].mean(),
        "team_total_shotsoff": team1Statistics["Shots off target"].sum(),
        "team_shots_blocked": team1Statistics["Shots blocked"].sum(),
        "team_deff_actions": team1Statistics["Defensive actions"].mean(),
        "team_clearances": team1Statistics["Clearances"].mean(),
        "team_blocked_shots": team1Statistics["Blocked shots"].sum(),
        "team_interceptions": team1Statistics["Interceptions"].sum(),
        "team_total_tackels": team1Statistics["Total tackels"].sum(),
        "team_dribbled_past": team1Statistics["Dribbled past"].sum(),
        "team_avg_touches": team1Statistics["Touches"].mean(),
        "team_avg_acc_pass_rating": team1Statistics["Accurate Pass Rating"].mean(),
        "team_avg_duel_rate": team1Statistics["Duel Win Rate"].mean(),
        "team_avg_ground_deul_rate": team1Statistics["Ground Duel Win Rate"].mean(),
        "team_avg_aerial_duel_rate": team1Statistics["Aerial Duel Win Rate"].mean(),
        "team_possession_lost_rate": team1Statistics["Possession Loss Rate"].mean(),
        "team_avg_succ_long_balls": team1Statistics["Successful Long Ball Rate"].mean(),
        "team_avg_succ_cross_rate": team1Statistics["Successful Cross Rate"].mean(),
        "team_avg_fouls_rate": team1Statistics["Foul Rate"].mean(),
        "team_total_key_passes": team1Statistics["Key passes"].sum()
    }

    team2Summary = {
        "team_total_goals": team2Statistics["Goals"].sum(),
        "team_total_assists": team2Statistics["Assists"].sum(),
        "team_total_shots": team2Statistics["Shots on target"].sum(),
        "team_avg_xG": team2Statistics["Expected goals (xG)"].mean(),
        "team_total_shotsoff": team2Statistics["Shots off target"].sum(),
        "team_shots_blocked": team2Statistics["Shots blocked"].sum(),
        "team_deff_actions": team2Statistics["Defensive actions"].mean(),
        "team_clearances": team2Statistics["Clearances"].mean(),
        "team_blocked_shots": team2Statistics["Blocked shots"].sum(),
        "team_interceptions": team2Statistics["Interceptions"].sum(),
        "team_total_tackels": team2Statistics["Total tackels"].sum(),
        "team_dribbled_past": team2Statistics["Dribbled past"].sum(),
        "team_avg_touches": team2Statistics["Touches"].mean(),
        "team_avg_acc_pass_rating": team2Statistics["Accurate Pass Rating"].mean(),
        "team_avg_duel_rate": team2Statistics["Duel Win Rate"].mean(),
        "team_avg_ground_deul_rate": team2Statistics["Ground Duel Win Rate"].mean(),
        "team_avg_aerial_duel_rate": team2Statistics["Aerial Duel Win Rate"].mean(),
        "team_possession_lost_rate": team2Statistics["Possession Loss Rate"].mean(),
        "team_avg_succ_long_balls": team2Statistics["Successful Long Ball Rate"].mean(),
        "team_avg_succ_cross_rate": team2Statistics["Successful Cross Rate"].mean(),
        "team_avg_fouls_rate": team2Statistics["Foul Rate"].mean(),
        "team_total_key_passes": team2Statistics["Key passes"].sum()
    }

    for _, row in matchData.iterrows():
        player = row["Player"]
        rowData = row[["Player", "MatchID"] + statisticsColumns].to_dict()

        if player in team1:
            rowData.update({
                "team_total_goals": team1Summary["team_total_goals"],
                "team_total_assists": team1Summary["team_total_assists"],
                "team_total_shots": team1Summary["team_total_shots"],
                "team_avg_xG": team1Summary["team_avg_xG"],
                "team_total_shotsoff": team1Summary["team_total_shotsoff"],
                "team_shots_blocked": team1Summary["team_shots_blocked"],
                "team_deff_actions": team1Summary["team_deff_actions"],
                "team_clearances": team1Summary["team_clearances"],
                "team_blocked_shots": team1Summary["team_blocked_shots"],
                "team_interceptions": team1Summary["team_interceptions"],
                "team_total_tackels": team1Summary["team_total_tackels"],
                "team_dribbled_past": team1Summary["team_dribbled_past"],
                "team_avg_touches": team1Summary["team_avg_touches"],
                "team_avg_acc_pass_rating": team1Summary["team_avg_acc_pass_rating"],
                "team_avg_duel_rate": team1Summary["team_avg_duel_rate"],
                "team_avg_ground_deul_rate": team1Summary["team_avg_ground_deul_rate"],
                "team_avg_aerial_duel_rate": team1Summary["team_avg_aerial_duel_rate"],
                "team_possession_lost_rate": team1Summary["team_possession_lost_rate"],
                "team_avg_succ_long_balls": team1Summary["team_avg_succ_long_balls"],
                "team_avg_succ_cross_rate": team1Summary["team_avg_succ_cross_rate"],
                "team_avg_fouls_rate": team1Summary["team_avg_fouls_rate"],
                "team_total_key_passes": team1Summary["team_total_key_passes"],
                "opp_team_total_goals": team2Summary["team_total_goals"],
                "opp_team_total_assists": team2Summary["team_total_assists"],
                "opp_team_total_shots": team2Summary["team_total_shots"],
                "opp_team_avg_xG": team2Summary["team_avg_xG"],
                "opp_team_total_shotsoff": team2Summary["team_total_shotsoff"],
                "opp_team_shots_blocked": team2Summary["team_shots_blocked"],
                "opp_team_deff_actions": team2Summary["team_deff_actions"],
                "opp_team_clearances": team2Summary["team_clearances"],
                "opp_team_blocked_shots": team2Summary["team_blocked_shots"],
                "opp_team_interceptions": team2Summary["team_interceptions"],
                "opp_team_total_tackels": team2Summary["team_total_tackels"],
                "opp_team_dribbled_past": team2Summary["team_dribbled_past"],
                "opp_team_avg_touches": team2Summary["team_avg_touches"],
                "opp_team_avg_acc_pass_rating": team2Summary["team_avg_acc_pass_rating"],
                "opp_team_avg_duel_rate": team2Summary["team_avg_duel_rate"],
                "opp_team_avg_ground_deul_rate": team2Summary["team_avg_ground_deul_rate"],
                "opp_team_avg_aerial_duel_rate": team2Summary["team_avg_aerial_duel_rate"],
                "opp_team_possession_lost_rate": team2Summary["team_possession_lost_rate"],
                "opp_team_avg_succ_long_balls": team2Summary["team_avg_succ_long_balls"],
                "opp_team_avg_succ_cross_rate": team2Summary["team_avg_succ_cross_rate"],
                "opp_team_avg_fouls_rate": team2Summary["team_avg_fouls_rate"],
                "opp_team_total_key_passes": team2Summary["team_total_key_passes"],
            })
        elif player in team2:
            rowData.update({
                "team_total_goals": team2Summary["team_total_goals"],
                "team_total_assists": team2Summary["team_total_assists"],
                "team_total_shots": team2Summary["team_total_shots"],
                "team_avg_xG": team2Summary["team_avg_xG"],
                "team_total_shotsoff": team2Summary["team_total_shotsoff"],
                "team_shots_blocked": team2Summary["team_shots_blocked"],
                "team_deff_actions": team2Summary["team_deff_actions"],
                "team_clearances": team2Summary["team_clearances"],
                "team_blocked_shots": team2Summary["team_blocked_shots"],
                "team_interceptions": team2Summary["team_interceptions"],
                "team_total_tackels": team2Summary["team_total_tackels"],
                "team_dribbled_past": team2Summary["team_dribbled_past"],
                "team_avg_touches": team2Summary["team_avg_touches"],
                "team_avg_acc_pass_rating": team2Summary["team_avg_acc_pass_rating"],
                "team_avg_duel_rate": team2Summary["team_avg_duel_rate"],
                "team_avg_ground_deul_rate": team2Summary["team_avg_ground_deul_rate"],
                "team_avg_aerial_duel_rate": team2Summary["team_avg_aerial_duel_rate"],
                "team_possession_lost_rate": team2Summary["team_possession_lost_rate"],
                "team_avg_succ_long_balls": team2Summary["team_avg_succ_long_balls"],
                "team_avg_succ_cross_rate": team2Summary["team_avg_succ_cross_rate"],
                "team_avg_fouls_rate": team2Summary["team_avg_fouls_rate"],
                "team_total_key_passes": team2Summary["team_total_key_passes"],
                "opp_team_total_goals": team1Summary["team_total_goals"],
                "opp_team_total_assists": team1Summary["team_total_assists"],
                "opp_team_total_shots": team1Summary["team_total_shots"],
                "opp_team_avg_xG": team1Summary["team_avg_xG"],
                "opp_team_total_shotsoff": team1Summary["team_total_shotsoff"],
                "opp_team_shots_blocked": team1Summary["team_shots_blocked"],
                "opp_team_deff_actions": team1Summary["team_deff_actions"],
                "opp_team_clearances": team1Summary["team_clearances"],
                "opp_team_blocked_shots": team1Summary["team_blocked_shots"],
                "opp_team_interceptions": team1Summary["team_interceptions"],
                "opp_team_total_tackels": team1Summary["team_total_tackels"],
                "opp_team_dribbled_past": team1Summary["team_dribbled_past"],
                "opp_team_avg_touches": team1Summary["team_avg_touches"],
                "opp_team_avg_acc_pass_rating": team1Summary["team_avg_acc_pass_rating"],
                "opp_team_avg_duel_rate": team1Summary["team_avg_duel_rate"],
                "opp_team_avg_ground_deul_rate": team1Summary["team_avg_ground_deul_rate"],
                "opp_team_avg_aerial_duel_rate": team1Summary["team_avg_aerial_duel_rate"],
                "opp_team_possession_lost_rate": team1Summary["team_possession_lost_rate"],
                "opp_team_avg_succ_long_balls": team1Summary["team_avg_succ_long_balls"],
                "opp_team_avg_succ_cross_rate": team1Summary["team_avg_succ_cross_rate"],
                "opp_team_avg_fouls_rate": team1Summary["team_avg_fouls_rate"],
                "opp_team_total_key_passes": team1Summary["team_total_key_passes"],
            })
        else:
            continue

        agregatedRows.append(rowData)

agregatedData = pd.DataFrame(agregatedRows)
agregatedData = agregatedData.fillna(0)

In [ ]:
print(agregatedData.info())


In [166]:
# Pozíció szerinti szétválogatás
defenderPlayers = agregatedData[agregatedData["Position_D"] == 1]
defenderPlayers = defenderPlayers.drop(columns = ["Position_D", "Position_M", "Position_F"])
midfielderPlayers = agregatedData[agregatedData["Position_M"] == 1]
midfielderPlayers = midfielderPlayers.drop(columns = ["Position_D", "Position_M", "Position_F"])
forwardPlayers = agregatedData[agregatedData["Position_F"] == 1]
forwardPlayers = forwardPlayers.drop(columns = ["Position_D", "Position_M", "Position_F"])

# Minden posztra kiválasztjuk a Rating (teljesítmény) értékeket
positionRatings = {
    "Defenders": defenderPlayers["Rating"],
    "Midfielders": midfielderPlayers["Rating"],
    "Forwards": forwardPlayers["Rating"]
}

In [ ]:
import joblib

positions = ['D', 'M', 'F']
model_names = ["ridge", "lasso", "elastic", "randomforest", "xgboost"]

results = []

for position in positions:
    position_col = f"Position_{position}"
    if position_col not in agregatedData.columns:
        continue

    val_subset = agregatedData[agregatedData[position_col] == 1]
    if val_subset.empty:
        continue

    feature_list = joblib.load(f"features_{position}.pkl")
    X_val = val_subset[feature_list].copy()
    y_val = val_subset["Rating"]

    # Végtelen és NaN értékek kezelése
    X_val.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_val.fillna(0, inplace=True)

    for model_name in model_names:
        model_path = f"model_{position}_{model_name}.pkl"
        try:
            model = joblib.load(model_path)

            # Ha a modell típus alapján skálázott bemenetet vár
            if model_name in ["ridge", "lasso", "elastic"]:
                try:
                    scaler = joblib.load(f"scaler_{position}.pkl")
                    X_val_scaled = scaler.transform(X_val)
                    y_pred = model.predict(X_val_scaled)
                except FileNotFoundError:
                    print(f"Skálázó hiányzik: scaler_{position}.pkl — a modell lehet, hogy nem működik helyesen")
                    y_pred = model.predict(X_val)
            else:
                y_pred = model.predict(X_val)

            # Metrikák
            mae = mean_absolute_error(y_val, y_pred)
            mape = mean_absolute_percentage_error(y_val, y_pred)
            r2 = r2_score(y_val, y_pred)

            # Cross-validation
            cv = KFold(n_splits = 55, shuffle = True, random_state = 42)
            try:
                cv_mae = -cross_val_score(model, X_val_scaled, y_val, cv = cv, scoring = 'neg_mean_absolute_error').mean()
                cv_r2 = cross_val_score(model, X_val_scaled, y_val, cv = cv, scoring = 'r2').mean()
            except Exception as e:
                print(f"CV hiba: {model_name} ({position}): {e}")
                cv_mae = None
                cv_r2 = None

            results.append({
                "Position": position,
                "Model": model_name,
                "MAE": round(mae, 3),
                "MAPE": round(mape, 3),
                "R2": round(r2, 3),
                "CV_MAE": round(cv_mae, 3) if cv_mae is not None else None,
                "CV_R2": round(cv_r2, 3) if cv_r2 is not None else None
            })
        except FileNotFoundError:
            print(f"Modell hiányzik: {model_path}")

# Eredmények táblázata
results_df = pd.DataFrame(results)
print(results_df)

VALIDÁCIÓS ADATHALMAZ TESZTELVE A NEURÁLIS HÁLÓN

In [168]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [169]:
excludeColumns = ["Player", "MathcID", "Rating"]
features = [col for col in agregatedData.columns if col not in excludeColumns]
X = agregatedData[features].copy()
y = agregatedData["Rating"]

In [170]:
# Hibás értékek kiszűrése
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(0, inplace=True)

# Skálázás és train-test split
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.1, random_state = 42)

In [171]:
# Tenzor konverzió
X_train_tensor = torch.tensor(X_train, dtype = torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype = torch.float).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype = torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype = torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size = 128, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 128)

In [172]:
# Neurális háló
class PlayerRatingNN(nn.Module):
    def __init__(self, input_dim):
        super(PlayerRatingNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.dropout1 = nn.Dropout(p=0.3)

        self.fc2 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(p=0.3)

        self.fc3 = nn.Linear(128, 64)
        self.dropout3 = nn.Dropout(p=0.3)

        self.fc4 = nn.Linear(64, 32)
        self.dropout4 = nn.Dropout(p=0.3)

        self.fc5 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)

        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)

        x = torch.relu(self.fc3(x))
        x = self.dropout3(x)

        x = torch.relu(self.fc4(x))
        x = self.dropout4(x)

        x = torch.sigmoid(self.fc5(x)) * 9 + 1  # [1,10] skálázás
        return x


    
model = PlayerRatingNN(input_dim = X_train.shape[1])

criterion = nn.MSELoss()    
optimizer = optim.Adam(model.parameters(), lr = 0.0005)

In [ ]:
# Tanítás
num_epochs = 1500
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test_tensor)
        val_loss = criterion(val_outputs, y_test_tensor).item()
    val_losses.append(val_loss)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}')

    torch.save(model.state_dict(), "player_rating_nn.pth")

In [ ]:
# Általános kiértékelés
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).numpy()
    y_true = y_test_tensor.numpy()

print("\n--- Általános teszthalmaz értékelés ---")
print(f"MAE:  {mean_absolute_error(y_true, y_pred):.3f}")
print(f"MAPE: {mean_absolute_percentage_error(y_true, y_pred):.3f}")
print(f"R2:   {r2_score(y_true, y_pred):.3f}")

# Pozícióspecifikus validáció
positions = ['D', 'M', 'F']
for pos in positions:
    print(f"\n--- Validáció pozíció szerint: {pos} ---")
    pos_col = f"Position_{pos}"
    if pos_col not in agregatedData.columns:
        print(f"Pozícióoszlop nem található: {pos_col}")
        continue

    val_subset = agregatedData[agregatedData[pos_col] == 1]
    if val_subset.empty:
        print("Nincs adat ebben a pozícióban.")
        continue

    X_val = val_subset[features].copy()
    y_val = val_subset["Rating"]

    # Hibás értékek eltávolítása
    X_val.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_val.fillna(0, inplace=True)

    X_val_scaled = scaler.transform(X_val)
    X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

    with torch.no_grad():
        y_val_pred = model(X_val_tensor).numpy()
        y_val_true = y_val_tensor.numpy()

    mae = mean_absolute_error(y_val_true, y_val_pred)
    mape = mean_absolute_percentage_error(y_val_true, y_val_pred)
    r2 = r2_score(y_val_true, y_val_pred)

    print(f"MAE:  {mae:.3f}")
    print(f"MAPE: {mape:.3f}")
    print(f"R2:   {r2:.3f}")

In [175]:
data = pd.read_csv('C:/Users/Tamás/Desktop/Gazdinfó/Allamvizsga_masodikfelvonas/test_Validation1.csv')

In [176]:
# Játékos oszlopok azonosítása
allPlayers = set(data["Player"].unique())
playerColumns = [col for col in data.columns if col in allPlayers]

statisticsColumns = data.columns.difference(playerColumns + ["MatchID", "Team", "Player", "Notes", "Defence notes", "Pass notes"])
statisticsColumns = [col for col in statisticsColumns if data[col].dtype in [int, float]]

# Meccsenként a játékosok szétválogatása csapatokra minden MatchID-re
teamInfoPerMatch = {}
allMatchIDs = data["MatchID"].unique()

In [177]:

for matchID in allMatchIDs:
    matchData = data[data["MatchID"] == matchID]
    team1 = set()
    team2 = set()
    playersInMatch = matchData["Player"].tolist()
    matchPlayerColumns = [col for col in data.columns if col in playersInMatch]

    for _, row in matchData.iterrows():
        playerName = row["Player"]
        for teamMate in matchPlayerColumns:
            value = row[teamMate]
            if value == matchID:
                team1.add(playerName)
            elif value == -matchID:
                team2.add(playerName)

    teamInfoPerMatch[matchID] = {
        "team1": sorted(team1),
        "team2": sorted(team2)
    }

    


In [178]:
# Aggregált csapatstatisztikák hozzárendelése minden játékoshoz
agregatedRows = []

for matchID in allMatchIDs:
    matchData = data[data["MatchID" ] == matchID]
    teams = teamInfoPerMatch[matchID]
    team1 = teams["team1"]
    team2 = teams["team2"]

    team1Statistics = matchData[matchData["Player"].isin(team1)][statisticsColumns]
    team2Statistics = matchData[matchData["Player"].isin(team2)][statisticsColumns]

    team1Summary = {
        "team_total_goals": team1Statistics["Goals"].sum(),
        "team_total_assists": team1Statistics["Assists"].sum(),
        "team_total_shots": team1Statistics["Shots on target"].sum(),
        "team_avg_xG": team1Statistics["Expected goals (xG)"].mean(),
        "team_total_shotsoff": team1Statistics["Shots off target"].sum(),
        "team_shots_blocked": team1Statistics["Shots blocked"].sum(),
        "team_deff_actions": team1Statistics["Defensive actions"].mean(),
        "team_clearances": team1Statistics["Clearances"].mean(),
        "team_blocked_shots": team1Statistics["Blocked shots"].sum(),
        "team_interceptions": team1Statistics["Interceptions"].sum(),
        "team_total_tackels": team1Statistics["Total tackels"].sum(),
        "team_dribbled_past": team1Statistics["Dribbled past"].sum(),
        "team_avg_touches": team1Statistics["Touches"].mean(),
        "team_avg_acc_pass_rating": team1Statistics["Accurate Pass Rating"].mean(),
        "team_avg_duel_rate": team1Statistics["Duel Win Rate"].mean(),
        "team_avg_ground_deul_rate": team1Statistics["Ground Duel Win Rate"].mean(),
        "team_avg_aerial_duel_rate": team1Statistics["Aerial Duel Win Rate"].mean(),
        "team_possession_lost_rate": team1Statistics["Possession Loss Rate"].mean(),
        "team_avg_succ_long_balls": team1Statistics["Successful Long Ball Rate"].mean(),
        "team_avg_succ_cross_rate": team1Statistics["Successful Cross Rate"].mean(),
        "team_avg_fouls_rate": team1Statistics["Foul Rate"].mean(),
        "team_total_key_passes": team1Statistics["Key passes"].sum()
    }

    team2Summary = {
        "team_total_goals": team2Statistics["Goals"].sum(),
        "team_total_assists": team2Statistics["Assists"].sum(),
        "team_total_shots": team2Statistics["Shots on target"].sum(),
        "team_avg_xG": team2Statistics["Expected goals (xG)"].mean(),
        "team_total_shotsoff": team2Statistics["Shots off target"].sum(),
        "team_shots_blocked": team2Statistics["Shots blocked"].sum(),
        "team_deff_actions": team2Statistics["Defensive actions"].mean(),
        "team_clearances": team2Statistics["Clearances"].mean(),
        "team_blocked_shots": team2Statistics["Blocked shots"].sum(),
        "team_interceptions": team2Statistics["Interceptions"].sum(),
        "team_total_tackels": team2Statistics["Total tackels"].sum(),
        "team_dribbled_past": team2Statistics["Dribbled past"].sum(),
        "team_avg_touches": team2Statistics["Touches"].mean(),
        "team_avg_acc_pass_rating": team2Statistics["Accurate Pass Rating"].mean(),
        "team_avg_duel_rate": team2Statistics["Duel Win Rate"].mean(),
        "team_avg_ground_deul_rate": team2Statistics["Ground Duel Win Rate"].mean(),
        "team_avg_aerial_duel_rate": team2Statistics["Aerial Duel Win Rate"].mean(),
        "team_possession_lost_rate": team2Statistics["Possession Loss Rate"].mean(),
        "team_avg_succ_long_balls": team2Statistics["Successful Long Ball Rate"].mean(),
        "team_avg_succ_cross_rate": team2Statistics["Successful Cross Rate"].mean(),
        "team_avg_fouls_rate": team2Statistics["Foul Rate"].mean(),
        "team_total_key_passes": team2Statistics["Key passes"].sum()
    }

    for _, row in matchData.iterrows():
        player = row["Player"]
        rowData = row[["Player", "MatchID"] + statisticsColumns].to_dict()

        if player in team1:
            rowData.update({
                "team_total_goals": team1Summary["team_total_goals"],
                "team_total_assists": team1Summary["team_total_assists"],
                "team_total_shots": team1Summary["team_total_shots"],
                "team_avg_xG": team1Summary["team_avg_xG"],
                "team_total_shotsoff": team1Summary["team_total_shotsoff"],
                "team_shots_blocked": team1Summary["team_shots_blocked"],
                "team_deff_actions": team1Summary["team_deff_actions"],
                "team_clearances": team1Summary["team_clearances"],
                "team_blocked_shots": team1Summary["team_blocked_shots"],
                "team_interceptions": team1Summary["team_interceptions"],
                "team_total_tackels": team1Summary["team_total_tackels"],
                "team_dribbled_past": team1Summary["team_dribbled_past"],
                "team_avg_touches": team1Summary["team_avg_touches"],
                "team_avg_acc_pass_rating": team1Summary["team_avg_acc_pass_rating"],
                "team_avg_duel_rate": team1Summary["team_avg_duel_rate"],
                "team_avg_ground_deul_rate": team1Summary["team_avg_ground_deul_rate"],
                "team_avg_aerial_duel_rate": team1Summary["team_avg_aerial_duel_rate"],
                "team_possession_lost_rate": team1Summary["team_possession_lost_rate"],
                "team_avg_succ_long_balls": team1Summary["team_avg_succ_long_balls"],
                "team_avg_succ_cross_rate": team1Summary["team_avg_succ_cross_rate"],
                "team_avg_fouls_rate": team1Summary["team_avg_fouls_rate"],
                "team_total_key_passes": team1Summary["team_total_key_passes"],
                "opp_team_total_goals": team2Summary["team_total_goals"],
                "opp_team_total_assists": team2Summary["team_total_assists"],
                "opp_team_total_shots": team2Summary["team_total_shots"],
                "opp_team_avg_xG": team2Summary["team_avg_xG"],
                "opp_team_total_shotsoff": team2Summary["team_total_shotsoff"],
                "opp_team_shots_blocked": team2Summary["team_shots_blocked"],
                "opp_team_deff_actions": team2Summary["team_deff_actions"],
                "opp_team_clearances": team2Summary["team_clearances"],
                "opp_team_blocked_shots": team2Summary["team_blocked_shots"],
                "opp_team_interceptions": team2Summary["team_interceptions"],
                "opp_team_total_tackels": team2Summary["team_total_tackels"],
                "opp_team_dribbled_past": team2Summary["team_dribbled_past"],
                "opp_team_avg_touches": team2Summary["team_avg_touches"],
                "opp_team_avg_acc_pass_rating": team2Summary["team_avg_acc_pass_rating"],
                "opp_team_avg_duel_rate": team2Summary["team_avg_duel_rate"],
                "opp_team_avg_ground_deul_rate": team2Summary["team_avg_ground_deul_rate"],
                "opp_team_avg_aerial_duel_rate": team2Summary["team_avg_aerial_duel_rate"],
                "opp_team_possession_lost_rate": team2Summary["team_possession_lost_rate"],
                "opp_team_avg_succ_long_balls": team2Summary["team_avg_succ_long_balls"],
                "opp_team_avg_succ_cross_rate": team2Summary["team_avg_succ_cross_rate"],
                "opp_team_avg_fouls_rate": team2Summary["team_avg_fouls_rate"],
                "opp_team_total_key_passes": team2Summary["team_total_key_passes"],
            })
        elif player in team2:
            rowData.update({
                "team_total_goals": team2Summary["team_total_goals"],
                "team_total_assists": team2Summary["team_total_assists"],
                "team_total_shots": team2Summary["team_total_shots"],
                "team_avg_xG": team2Summary["team_avg_xG"],
                "team_total_shotsoff": team2Summary["team_total_shotsoff"],
                "team_shots_blocked": team2Summary["team_shots_blocked"],
                "team_deff_actions": team2Summary["team_deff_actions"],
                "team_clearances": team2Summary["team_clearances"],
                "team_blocked_shots": team2Summary["team_blocked_shots"],
                "team_interceptions": team2Summary["team_interceptions"],
                "team_total_tackels": team2Summary["team_total_tackels"],
                "team_dribbled_past": team2Summary["team_dribbled_past"],
                "team_avg_touches": team2Summary["team_avg_touches"],
                "team_avg_acc_pass_rating": team2Summary["team_avg_acc_pass_rating"],
                "team_avg_duel_rate": team2Summary["team_avg_duel_rate"],
                "team_avg_ground_deul_rate": team2Summary["team_avg_ground_deul_rate"],
                "team_avg_aerial_duel_rate": team2Summary["team_avg_aerial_duel_rate"],
                "team_possession_lost_rate": team2Summary["team_possession_lost_rate"],
                "team_avg_succ_long_balls": team2Summary["team_avg_succ_long_balls"],
                "team_avg_succ_cross_rate": team2Summary["team_avg_succ_cross_rate"],
                "team_avg_fouls_rate": team2Summary["team_avg_fouls_rate"],
                "team_total_key_passes": team2Summary["team_total_key_passes"],
                "opp_team_total_goals": team1Summary["team_total_goals"],
                "opp_team_total_assists": team1Summary["team_total_assists"],
                "opp_team_total_shots": team1Summary["team_total_shots"],
                "opp_team_avg_xG": team1Summary["team_avg_xG"],
                "opp_team_total_shotsoff": team1Summary["team_total_shotsoff"],
                "opp_team_shots_blocked": team1Summary["team_shots_blocked"],
                "opp_team_deff_actions": team1Summary["team_deff_actions"],
                "opp_team_clearances": team1Summary["team_clearances"],
                "opp_team_blocked_shots": team1Summary["team_blocked_shots"],
                "opp_team_interceptions": team1Summary["team_interceptions"],
                "opp_team_total_tackels": team1Summary["team_total_tackels"],
                "opp_team_dribbled_past": team1Summary["team_dribbled_past"],
                "opp_team_avg_touches": team1Summary["team_avg_touches"],
                "opp_team_avg_acc_pass_rating": team1Summary["team_avg_acc_pass_rating"],
                "opp_team_avg_duel_rate": team1Summary["team_avg_duel_rate"],
                "opp_team_avg_ground_deul_rate": team1Summary["team_avg_ground_deul_rate"],
                "opp_team_avg_aerial_duel_rate": team1Summary["team_avg_aerial_duel_rate"],
                "opp_team_possession_lost_rate": team1Summary["team_possession_lost_rate"],
                "opp_team_avg_succ_long_balls": team1Summary["team_avg_succ_long_balls"],
                "opp_team_avg_succ_cross_rate": team1Summary["team_avg_succ_cross_rate"],
                "opp_team_avg_fouls_rate": team1Summary["team_avg_fouls_rate"],
                "opp_team_total_key_passes": team1Summary["team_total_key_passes"],
            })
        else:
            continue

        agregatedRows.append(rowData)


In [179]:
agregatedTestData = pd.DataFrame(agregatedRows)
agregatedTestData = agregatedTestData.fillna(0)

In [ ]:
print(agregatedTestData.info())

In [181]:
# Feature oszlopok az új adatból
exclude_columns = ["Player", "MatchID"]
feature_columns = [col for col in agregatedTestData.columns if col not in exclude_columns]

X_new = agregatedTestData[feature_columns].copy()
X_new.replace([np.inf, -np.inf], np.nan, inplace=True)
X_new.fillna(0, inplace=True)

In [182]:
# Skálázás: most újra létrehozzuk ugyanazzal a fit logikával, mint korábban
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)  # mivel nincs mentett scaler

In [183]:
# Modell betöltése (újra létrehozva, mert nem lett mentve)
model = PlayerRatingNN(input_dim=X_scaled.shape[1])
model.load_state_dict(torch.load("player_rating_nn.pth"))
model.eval()

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

with torch.no_grad():
    predicted_ratings = model(X_tensor).numpy().flatten()


In [184]:
# Mesterségesen újratanítva nem volt, ezért most feltöltetlen modell lesz
model.eval()
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

with torch.no_grad():
    predicted_ratings = model(X_tensor).numpy().flatten()

In [185]:
# Eredmények hozzárendelése
agregatedTestData["Predicted Rating"] = predicted_ratings

In [ ]:
print(agregatedTestData[["Player", "Predicted Rating"]].head(30))